1. ADPC POUR LES DATASET REDUITS 

In [2]:
# Importer des modules
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import time

def calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c):
    # Initialisation des variables
    num_time_steps = len(power_consumption_sequence)
    rho_values = np.zeros(num_time_steps)
    delta_values = np.zeros(num_time_steps)

    # Calcul des distances entre toutes les paires de valeurs de consommation d'énergie
    distance_matrix = cdist(power_consumption_sequence.reshape(-1, 1), power_consumption_sequence.reshape(-1, 1), metric='euclidean')
    
    # Calcul des valeurs rho
    for i in range(1, num_time_steps):
        for j in range(i + 1, num_time_steps):
            rho_values[i] += chi(distance_matrix[i, j] - d_c)
            rho_values[j] += chi(distance_matrix[i, j] - d_c)

    # Trier les indices par valeurs rho dans l'ordre décroissant
    sorted_indices = np.argsort(-rho_values)

    # Calcul des valeurs delta
    delta_values.fill(np.max(distance_matrix) * num_time_steps)
    delta_values[sorted_indices[0]] = -1
    for i in range(1, num_time_steps):
        for j in range(i):
            if distance_matrix[sorted_indices[i], sorted_indices[j]] < delta_values[sorted_indices[i]]:
                delta_values[sorted_indices[i]] = distance_matrix[sorted_indices[i], sorted_indices[j]]

    delta_values[sorted_indices[0]] = np.max(delta_values)

    # Déterminer la séquence de puissance correspondant aux différents états de fonctionnement de l'appareil
    power_sequence = [power_consumption_sequence[i] for i in range(num_time_steps) if rho_values[i] > rho_threshold and delta_values[i] > delta_threshold]
    num_working_states = len(power_sequence)

    return sorted(power_sequence), num_working_states  # Tri des valeurs avant de les retourner


def chi(x) :
    
    return x 


# Liste des noms de fichiers CSV à lire
file_names = ['boiler_226_reduits.csv', 'computer_44_reduits.csv', 'dishwasher_53_reduits.csv', 'dryer_219_reduits.csv', 'fridge_284_reduits.csv', 'laptop_289_reduits.csv', 'tv_290_reduits.csv', 'vacuum_236_reduits.csv', 'wm_343_reduits.csv']

# Créer un DataFrame vide pour stocker les résultats
results_df = pd.DataFrame(columns=["device", "real_power", "num_states"])

# Mesurer le temps d'exécution total
total_start_time = time.time()

# Boucle à travers chaque fichier
for file_name in file_names:
    # Lire le fichier CSV dans un DataFrame
    df = pd.read_csv(file_name)
    
    # Obtenir la séquence de consommation d'énergie
    power_consumption_sequence = df['power'].values
    
    # Définissez vos paramètres spécifiques ici
    rho_threshold = 0.01 * (len(power_consumption_sequence) - max(power_consumption_sequence))
    delta_threshold = 0.1 * (max(power_consumption_sequence))
    d_c = 5
    
    # Mesurer le temps d'exécution pour chaque itération
    start_time = time.time()
    
    # Appeler la fonction calculate_power_sequence avec vos paramètres
    power_sequence, num_working_states = calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
    
    # Ajouter les résultats au DataFrame
    results_df = pd.concat([results_df, pd.DataFrame({"device": [file_name.split('_')[0]], 
                                                       "real_power": [power_sequence], 
                                                       "num_states": [num_working_states]})], ignore_index=True)

# Trier les valeurs à l'intérieur de chaque liste de résultats par ordre croissant
results_df['real_power'] = results_df['real_power'].apply(sorted)

# Mesurer le temps d'exécution total
total_execution_time = time.time() - total_start_time

# Stocker les résultats dans un nouveau fichier CSV
results_df.to_csv("dataset_appliances.csv", index=False)

# Afficher le DataFrame
print(results_df)

# Signes de démarcation
print('='*60)

# Afficher le temps d'exécution total
print(f"Total computational time: {total_execution_time:.4f} seconds")


       device                     real_power num_states
0      boiler                             []          0
1    computer             [2.0, 62.0, 280.0]          3
2  dishwasher                             []          0
3       dryer                  [0.0, 3267.0]          2
4      fridge            [1.0, 113.0, 280.0]          3
5      laptop                    [0.0, 61.0]          2
6          tv                    [1.0, 54.0]          2
7      vacuum  [207.0, 549.5, 839.0, 1099.0]          4
8          wm       [1459.5, 1961.0, 2663.0]          3
Total computational time: 14.4504 seconds


2. FORMER DES HMM POUR CHAQUE APPAREIL

In [3]:
import numpy as np
import pandas as pd
import re

class HMM:
    def __init__(self, num_states, num_obs):
        # Initialisation des paramètres du HMM avec des valeurs par défaut
        self.num_states = num_states                                                # Nombre d'états cachés du HMM
        self.num_obs = num_obs                                                     # Nombre d'observations possibles
        self.initial_prob = np.ones(num_states) / num_states                      # Probabilités initiales des états cachés
        self.transition_prob = np.ones((num_states, num_states)) / num_states    # Probabilités de transition entre les états cachés
        self.emission_prob = np.ones((num_states, num_obs)) / num_obs           # Probabilités d'émission des observations depuis les états cachés

    # Autres méthodes de la classe HMM pour le lissage forward-backward, le calcul des probabilités forward et backward, etc.

# Fonction pour créer un HMM pour chaque appareil
def create_hmm(appliances_info):
    hmms = {}
    for appliance, info in appliances_info.items():
        num_states = info['num_states']
        num_obs = info['num_obs']
        hmm = HMM(num_states=num_states, num_obs=num_obs)
        hmms[appliance] = hmm
    return hmms

# Fonction pour lire les informations sur les appareils à partir d'un fichier CSV
def read_appliances_info(csv_file):
    appliances_info = {}
    df = pd.read_csv(csv_file)
    for index, row in df.iterrows():
        appliance = row['device']
        num_states = row['num_states']
        real_power = row['real_power']

        # Convertir la chaîne de caractères entre crochets en une liste de nombres
        power_values = re.findall(r'\d+', real_power)
        # Compter le nombre d'éléments dans la liste pour obtenir le nombre d'observations possibles
        num_obs = len(power_values)

        appliances_info[appliance] = {'num_states': num_states, 'num_obs': num_obs}
    return appliances_info

# Exemple d'utilisation :
csv_file = "dataset_appliances.csv"
appliances_info = read_appliances_info(csv_file)
hmms = create_hmm(appliances_info)

# Affichage des HMM créés pour chaque appareil
print("HMM créés pour chaque appareil :")
for appliance, hmm in hmms.items():
    print(f"Appareil : {appliance}")
    print(f"Nombre d'états cachés : {hmm.num_states}")
    print(f"Nombre d'observations possibles : {hmm.num_obs}")
    print(f"Probabilités initiales des états cachés : {hmm.initial_prob}")
    print(f"Probabilités de transition entre les états cachés : {hmm.transition_prob}")
    print(f"Probabilités d'émission des observations depuis les états cachés : {hmm.emission_prob}")
    print()


HMM créés pour chaque appareil :
Appareil : boiler
Nombre d'états cachés : 0
Nombre d'observations possibles : 0
Probabilités initiales des états cachés : []
Probabilités de transition entre les états cachés : []
Probabilités d'émission des observations depuis les états cachés : []

Appareil : computer
Nombre d'états cachés : 3
Nombre d'observations possibles : 6
Probabilités initiales des états cachés : [0.33333333 0.33333333 0.33333333]
Probabilités de transition entre les états cachés : [[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]
Probabilités d'émission des observations depuis les états cachés : [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Appareil : dishwasher
Nombre d'états cachés : 0
Nombre d'observations possibles : 0
Probabilités initiales des états cac

3. COMBINER TOUS LES HMM POUR FORMER UN FHMM

In [6]:
import numpy as np

# Définition des HMM individuels
hmm_computer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_dryer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_fridge = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_laptop = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_tv = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_vacuum = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_wm = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }


# Fonction pour combiner les HMM individuels en un seul FHMM
def combine_hmms(hmms):
    num_states = max([len(hmm['init_prob']) for hmm in hmms])
    init_probs = np.zeros(num_states)
    trans_probs = np.zeros((num_states, num_states))
    emit_probs = None  # La forme des probabilités d'émission dépendra du nombre d'observations possibles

    for hmm in hmms:
        init_probs[:len(hmm['init_prob'])] += hmm['init_prob']
        trans_probs[:len(hmm['trans_prob']), :len(hmm['trans_prob'])] += hmm['trans_prob']

        if emit_probs is None:
            emit_probs = np.zeros((num_states, len(hmm['emit_prob'][0])))
        emit_probs[:len(hmm['emit_prob']), :] += hmm['emit_prob']

    # Normalisation des probabilités
    init_probs /= len(hmms)
    trans_probs /= len(hmms)
    emit_probs /= len(hmms)

    return {'init_prob': init_probs, 'trans_prob': trans_probs, 'emit_prob': emit_probs}

# Liste des HMM individuels
hmms = [hmm_computer, hmm_dryer, hmm_fridge, hmm_laptop, hmm_tv, hmm_vacuum, hmm_wm] 

# Création du FHMM en combinant les HMM individuels
fhmm = combine_hmms(hmms)

# Affichage des paramètres du FHMM
print("Paramètres du FHMM combiné :")
print("Probabilités initiales :", fhmm['init_prob'])
print("Probabilités de transition :", fhmm['trans_prob'])
print("Probabilités d'émission :", fhmm['emit_prob'])


IndexError: list index out of range

In [ ]:
import numpy as np

# Définition des HMM individuels
hmm_computer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_dryer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_fridge = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_laptop = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_tv = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_vacuum = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_wm = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }



# Liste des HMM individuels
hmms = [hmm_boiler, hmm_computer, hmm_dishwasher, hmm_dryer, hmm_fridge, hmm_laptop, hmm_tv, hmm_vacuum, hmm_wm]

# Création du FHMM en combinant les HMM individuels
fhmm = combine_hmms(hmms)

# Affichage des paramètres du FHMM
print("Paramètres du FHMM combiné :")
print("Probabilités initiales :", fhmm['init_prob'])
print("Probabilités de transition :", fhmm['trans_prob'])
print("Probabilités d'émission :", fhmm['emit_prob'])


In [7]:
import numpy as np

# Définition des HMM individuels
hmm_computer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_dryer = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_fridge = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_laptop = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_tv = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_vacuum = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }
hmm_wm = { 'init_prob': [], 'trans_prob': [], 'emit_prob': [] }


def combine_hmms(hmms):
    num_states = max([len(hmm['init_prob']) for hmm in hmms])
    init_probs = np.zeros(num_states)
    trans_probs = np.zeros((num_states, num_states))
    emit_probs = None

    for hmm in hmms:
        init_probs[:len(hmm['init_prob'])] += hmm['init_prob']
        trans_probs[:len(hmm['trans_prob']), :len(hmm['trans_prob'])] += hmm['trans_prob']

        if 'emit_prob' in hmm and len(hmm['emit_prob']) > 0:
            if emit_probs is None:
                emit_probs = np.zeros((num_states, len(hmm['emit_prob'][0])))
            emit_probs[:len(hmm['emit_prob']), :] += hmm['emit_prob']

    init_probs /= len(hmms)
    trans_probs /= len(hmms)
    if emit_probs is not None:
        emit_probs /= len(hmms)

    return {'init_prob': init_probs, 'trans_prob': trans_probs, 'emit_prob': emit_probs}

# Liste des HMM individuels
hmms = [hmm_computer, hmm_dryer, hmm_fridge, hmm_laptop, hmm_tv, hmm_vacuum, hmm_wm] 

# Création du FHMM en combinant les HMM individuels
fhmm = combine_hmms(hmms)

# Affichage des paramètres du FHMM
print("Paramètres du FHMM combiné :")
print("Probabilités initiales :", fhmm['init_prob'])
print("Probabilités de transition :", fhmm['trans_prob'])
print("Probabilités d'émission :", fhmm['emit_prob'])



Paramètres du FHMM combiné :
Probabilités initiales : []
Probabilités de transition : []
Probabilités d'émission : None


In [ ]:
import numpy as np

# Définition des HMM individuels
hmm_computer = {
    'init_prob': np.array([0.33333333, 0.33333333, 0.33333333]),
    'trans_prob': np.array([[0.33333333, 0.33333333, 0.33333333],
                            [0.33333333, 0.33333333, 0.33333333],
                            [0.33333333, 0.33333333, 0.33333333]]),
    'emit_prob': np.array([[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                           [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                           [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]])
}

hmm_dryer = {
    'init_prob': np.array([0.5, 0.5]),
    'trans_prob': np.array([[0.5, 0.5], [0.5, 0.5]]),
    'emit_prob': np.array([[0.25, 0.25, 0.25, 0.25],
                           [0.25, 0.25, 0.25, 0.25]])
}

# Définition de la fonction pour combiner les HMM en un seul FHMM
def combine_hmms(hmms):
    num_states = max(max(len(hmm['init_prob']), len(hmm['trans_prob'])) for hmm in hmms)
    num_observations = max(max(len(prob) for prob in hmm['emit_prob']) for hmm in hmms if hmm['emit_prob'] is not None)
    init_probs = np.zeros(num_states)
    trans_probs = np.zeros((num_states, num_states))
    emit_probs_sum = np.zeros((num_states, num_observations)) if num_observations > 0 else None

    total_hmms = len(hmms)

    for hmm in hmms:
        init_probs[:len(hmm['init_prob'])] += hmm['init_prob']
        trans_probs[:len(hmm['trans_prob']), :len(hmm['trans_prob'])] += hmm['trans_prob']

        if hmm['emit_prob'] is not None:
            emit_probs_sum[:len(hmm['emit_prob']), :len(hmm['emit_prob'][0])] += hmm['emit_prob']

    init_probs /= total_hmms
    trans_probs /= total_hmms

    emit_probs = emit_probs_sum / total_hmms if emit_probs_sum is not None else None

    return {'init_prob': init_probs, 'trans_prob': trans_probs, 'emit_prob': emit_probs}

# Liste des HMM individuels
hmms = [hmm_computer, hmm_dryer]  # Ajoutez les autres HMM ici

# Création du FHMM en combinant les HMM individuels
fhmm = combine_hmms(hmms)

# Affichage des paramètres du FHMM
print("Paramètres du FHMM combiné :")
print("Probabilités initiales :", fhmm['init_prob'])
print("Probabilités de transition :", fhmm['trans_prob'])
print("Probabilités d'émission :", fhmm['emit_prob'])


In [12]:
import pandas as pd

# Définition des HMM individuels
hmm_computer = {
    'appareil': 'computer',
    'nombre_etats': 3,
    'nombre_observations': 6,
    'prob_initiales': [0.33333333, 0.33333333, 0.33333333],
    'prob_transitions': [[0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333]],
    'prob_emissions': [[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]]
}

hmm_dryer = {
    'appareil': 'dryer',
    'nombre_etats': 2,
    'nombre_observations': 4,
    'prob_initiales': [0.5, 0.5],
    'prob_transitions': [[0.5, 0.5], [0.5, 0.5]],
    'prob_emissions': [[0.25, 0.25, 0.25, 0.25],
                       [0.25, 0.25, 0.25, 0.25]]
}

hmm_fridge = {
    'appareil': 'fridge',
    'nombre_etats': 3,
    'nombre_observations': 6,
    'prob_initiales': [0.33333333, 0.33333333, 0.33333333],
    'prob_transitions': [[0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333]],
    'prob_emissions': [[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]]
}

hmm_laptop = {
    'appareil': 'laptop',
    'nombre_etats': 2,
    'nombre_observations': 4,
    'prob_initiales': [0.5, 0.5],
    'prob_transitions': [[0.5, 0.5], [0.5, 0.5]],
    'prob_emissions': [[0.25, 0.25, 0.25, 0.25],
                       [0.25, 0.25, 0.25, 0.25]]
}

hmm_tv = {
    'appareil': 'tv',
    'nombre_etats': 2,
    'nombre_observations': 4,
    'prob_initiales': [0.5, 0.5],
    'prob_transitions': [[0.5, 0.5], [0.5, 0.5]],
    'prob_emissions': [[0.25, 0.25, 0.25, 0.25],
                       [0.25, 0.25, 0.25, 0.25]]
}

hmm_vacuum = {
    'appareil': 'vacuum',
    'nombre_etats': 4,
    'nombre_observations': 8,
    'prob_initiales': [0.25, 0.25, 0.25, 0.25],
    'prob_transitions': [[0.25, 0.25, 0.25, 0.25],
                         [0.25, 0.25, 0.25, 0.25],
                         [0.25, 0.25, 0.25, 0.25],
                         [0.25, 0.25,0.25, 0.25]],
    'prob_emissions': [[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
                       [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
                       [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
                       [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]]
}

hmm_wm = {
    'appareil': 'fridge',
    'nombre_etats': 3,
    'nombre_observations': 6,
    'prob_initiales': [0.33333333, 0.33333333, 0.33333333],
    'prob_transitions': [[0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333],
                         [0.33333333, 0.33333333, 0.33333333]],
    'prob_emissions': [[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
                       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]]
}


# Création d'un DataFrame à partir des HMM individuels
hmm_data = [hmm_computer, hmm_dryer, hmm_fridge, hmm_laptop, hmm_tv, hmm_vacuum, hmm_wm] 

df_hmm = pd.DataFrame(hmm_data)

# Stockage du DataFrame dans un fichier CSV
df_hmm.to_csv('hmm_data.csv', index=False)

# Chargement du DataFrame à partir du fichier CSV
df_hmm_loaded = pd.read_csv('hmm_data.csv')
print(df_hmm_loaded)


   appareil  nombre_etats  nombre_observations  \
0  computer             3                    6   
1     dryer             2                    4   
2    fridge             3                    6   
3    laptop             2                    4   
4        tv             2                    4   
5    vacuum             4                    8   
6    fridge             3                    6   

                         prob_initiales  \
0  [0.33333333, 0.33333333, 0.33333333]   
1                            [0.5, 0.5]   
2  [0.33333333, 0.33333333, 0.33333333]   
3                            [0.5, 0.5]   
4                            [0.5, 0.5]   
5              [0.25, 0.25, 0.25, 0.25]   
6  [0.33333333, 0.33333333, 0.33333333]   

                                    prob_transitions  \
0  [[0.33333333, 0.33333333, 0.33333333], [0.3333...   
1                           [[0.5, 0.5], [0.5, 0.5]]   
2  [[0.33333333, 0.33333333, 0.33333333], [0.3333...   
3                           [[

In [16]:
import numpy as np
import pandas as pd
import ast  # Module pour analyser une chaîne de caractères en une structure de données Python

# Fonction pour combiner les HMM individuels en un seul FHMM
def combine_hmms(hmms):
    num_states = max([len(ast.literal_eval(hmm['prob_initiales'])) for hmm in hmms])
    num_observations = max([len(prob) for hmm in hmms for prob in ast.literal_eval(hmm['prob_emissions'])]) if any(hmm['prob_emissions'] for hmm in hmms) else 0
    init_probs = np.zeros(num_states)
    trans_probs = np.zeros((num_states, num_states))
    emit_probs_sum = np.zeros((num_states, num_observations))  # Nous allons d'abord accumuler les émissions

    total_hmms = len(hmms)

    for hmm in hmms:
        # Conversion des probabilités initiales en listes de nombres
        init_prob_float = np.array(ast.literal_eval(hmm['prob_initiales']), dtype=float)
        # Agrégation des probabilités initiales
        init_probs[:len(init_prob_float)] += init_prob_float

        # Conversion des probabilités de transition en listes de listes de nombres
        trans_prob_float = np.array(ast.literal_eval(hmm['prob_transitions']), dtype=float)
        # Agrégation des probabilités de transition
        trans_probs[:len(trans_prob_float), :len(trans_prob_float)] += trans_prob_float

        # Agrégation des probabilités d'émission
        if hmm['prob_emissions'] is not None:
            emit_prob_float = np.array(ast.literal_eval(hmm['prob_emissions']), dtype=float)
            num_obs_hmm = len(emit_prob_float[0])
            emit_probs_sum[:len(emit_prob_float), :num_obs_hmm] += emit_prob_float

    # Normalisation des probabilités
    init_probs /= total_hmms
    trans_probs /= total_hmms

    # Assurons-nous qu'il y a au moins un HMM avec des probabilités d'émission
    if np.any(emit_probs_sum):
        emit_probs = emit_probs_sum / total_hmms
    else:
        emit_probs = None

    return {'prob_initiales': init_probs, 'prob_transitions': trans_probs, 'prob_emissions': emit_probs}

# Chargement du DataFrame à partir du fichier CSV
df_hmm_loaded = pd.read_csv('hmm_data.csv')

# Création du FHMM en combinant les HMM individuels
fhmm = combine_hmms(df_hmm_loaded.to_dict(orient='records'))

# Affichage des paramètres du FHMM
print("Paramètres du FHMM combiné :")
print("Probabilités initiales :", fhmm['prob_initiales'])
print("Probabilités de transition :", fhmm['prob_transitions'])
print("Probabilités d'émission :", fhmm['prob_emissions'])


Paramètres du FHMM combiné :
Probabilités initiales : [0.39285714 0.39285714 0.17857143 0.03571429]
Probabilités de transition : [[0.39285714 0.39285714 0.17857143 0.03571429]
 [0.39285714 0.39285714 0.17857143 0.03571429]
 [0.17857143 0.17857143 0.17857143 0.03571429]
 [0.03571429 0.03571429 0.03571429 0.03571429]]
Probabilités d'émission : [[0.19642857 0.19642857 0.19642857 0.19642857 0.08928572 0.08928572
  0.01785714 0.01785714]
 [0.19642857 0.19642857 0.19642857 0.19642857 0.08928572 0.08928572
  0.01785714 0.01785714]
 [0.08928572 0.08928572 0.08928572 0.08928572 0.08928572 0.08928572
  0.01785714 0.01785714]
 [0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
  0.01785714 0.01785714]]


In [18]:
# Calcul du nombre d'états cachés combinés du FHMM
nb_etats_combines = np.count_nonzero(fhmm['prob_initiales'])
print("Nombre d'états cachés combinés du FHMM :", nb_etats_combines)


Nombre d'états cachés combinés du FHMM : 4


POWER MAPPING PROCESS

In [22]:
"""
Code acceptable algorithme cartographie de puissance
Acceptable code for power mapping process algorithm

"""
# Define power mapping function 
def calculate_real_power(mu, CZ, WS):
    N = len(mu)  # Number of appliances
    
    # Initialize the real power consumption array
    RP = [[0]*N for _ in range(len(CZ))]
    AS = [[0]*N for _ in range(len(CZ))]

    # Iterate over each combined hidden state
    for t, z in enumerate(CZ):
        # Iterate over each appliance
        for i in range(N):
            # Calculate the activation state
            AS[t][i] = int((z * prod(WS[:i+1]) / prod(WS)) % WS[i])
            # Calculate the real power consumption
            RP[t][i] = mu[i][AS[t][i]]

    return AS, RP

def prod(lst):
    # Helper function to calculate the product of a list
    result = 1
    for x in lst:
        result *= x
    return result

# Define your specific parameters here
mu = [[2.0, 62.0, 280.0], [0.0, 3267.0], [1.0, 113.0, 280.0], [0.0, 61.0], [1.0, 54.0], [207.0, 549.5, 839.0, 1099.0], [1459.5, 1961.0, 2663.0]]  # Power consumption of each appliance
CZ = [1200]  # Combined hidden state obtained by FHMM model
WS = [3, 2, 3, 2, 2, 4, 3]  # Number of operating states for each appliance

# Call the calculate_real_power function with your parameters
AS, RP = calculate_real_power(mu, CZ, WS)

# Print the result
for t in range(len(CZ)):
    print(f"Time {t}:")
    for i in range(len(mu)):
        print(f"  Appliance {i+1}: AS = {AS[t][i]}, RP = {RP[t][i]} W")
        

Time 0:
  Appliance 1: AS = 1, RP = 62.0 W
  Appliance 2: AS = 0, RP = 0.0 W
  Appliance 3: AS = 1, RP = 113.0 W
  Appliance 4: AS = 0, RP = 0.0 W
  Appliance 5: AS = 0, RP = 1.0 W
  Appliance 6: AS = 0, RP = 207.0 W
  Appliance 7: AS = 0, RP = 1459.5 W
